## Training an IRT model with py-irt

This document walks through using the py-irt package.

First, you need to make sure that py-irt is installed.

In [1]:
!pip install py-irt

  Using cached py_irt-0.4.4-py3-none-any.whl (26 kB)
     |████████████████████████████████| 23.8 MB 18.0 MB/s eta 0:00:01
  Using cached typer-0.3.2-py3-none-any.whl (21 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
     |████████████████████████████████| 12.4 MB 9.8 MB/s eta 0:00:01
  Using cached rich-9.13.0-py3-none-any.whl (197 kB)
  Using cached pyro_ppl-1.8.1-py3-none-any.whl (718 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
     |████████████████████████████████| 750.6 MB 17 kB/s s eta 0:00:01     |████████████▏                   | 286.6 MB 8.2 MB/s eta 0:00:57     |███████████████████▊            | 462.4 MB 11.4 MB/s eta 0:00:26     |█████████████████████           | 494.0 MB 11.3 MB/s eta 0:00:23
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Using cached 

Then, download the sample data file, stored as a jsonlines document.

In [2]:
!wget https://raw.githubusercontent.com/nd-ball/py-irt/d2a27dd55a84459782a5514e752ee48d9a63626e/test_fixtures/minitest.jsonlines

--2022-06-20 14:41:44--  https://raw.githubusercontent.com/nd-ball/py-irt/d2a27dd55a84459782a5514e752ee48d9a63626e/test_fixtures/minitest.jsonlines
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 311 [text/plain]
Saving to: ‘minitest.jsonlines’

minitest.jsonlines  100%[===================>]     311  --.-KB/s    in 0s      

2022-06-20 14:41:46 (6.49 MB/s) - ‘minitest.jsonlines’ saved [311/311]



In [3]:
!cat minitest.jsonlines

{"subject_id": "pedro",    "responses": {"q1": 1, "q2": 0, "q3": 1, "q4": 0}}
{"subject_id": "pinguino", "responses": {"q1": 1, "q2": 1, "q3": 0, "q4": 0}}
{"subject_id": "ken",      "responses": {"q1": 1, "q2": 1, "q3": 1, "q4": 1}}
{"subject_id": "burt",     "responses": {"q1": 0, "q2": 0, "q3": 0, "q4": 0}}

We'll first look at the options available to us in the py-irt package for the **train** command.
In this case, because the dataset is so small, we'll use a smaller learning rate and train for a smaller number of epochs than the defaults specify.

In [5]:
!py-irt train --help

Usage: py-irt train [OPTIONS] MODEL_TYPE DATA_PATH OUTPUT_DIR

Arguments:
  MODEL_TYPE  [required]
  DATA_PATH   [required]
  OUTPUT_DIR  [required]

Options:
  --epochs INTEGER
  --priors TEXT
  --dims INTEGER
  --lr FLOAT
  --lr-decay FLOAT
  --device TEXT        [default: cpu]
  --initializers TEXT
  --config-path TEXT
  --dropout FLOAT      [default: 0.5]
  --hidden INTEGER     [default: 100]
  --help               Show this message and exit.


We then run the py-irt package to fit a 1PL model and save it to a specified output directory.

In [6]:
!py-irt train 1pl minitest.jsonlines test-1pl/ --lr 0.02 --epochs 100

[14:43:39] config: model_type='1pl' epochs=100 priors=None             ]8;id=1655750619.4997613-658926;file:///home/lalor/miniconda3/envs/pedro/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:66
           initializers=[] dims=None lr=0.02 lr_decay=0.9999                    
           dropout=0.5 hidden=100 vocab_size=None                               
           data_path: minitest.jsonlines                               ]8;id=1655750619.5006726-866513;file:///home/lalor/miniconda3/envs/pedro/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:68
           output directory: test-1pl/                                 ]8;id=1655750619.5016015-82202;file:///home/lalor/miniconda3/envs/pedro/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:69
False
None
           Training Model...                                           ]8;id=1655750619.502791-293357;file:///home/lalor/miniconda3/envs/pedro/lib/python3.9/site-packages/py_irt/cli.py\cli.py]8;;\:73
{'device'

We can inspect the best fitting parameters and the parameters from the final training epoch.
To make it look nice we can use jq.

In [8]:
!pip install jq

     |████████████████████████████████| 591 kB 123 kB/s eta 0:00:01


In [9]:
!jq . test-1pl/best_parameters.json

{
  "ability": [
    0.202667236328125,
    -0.95220547914505,
    0.9865020513534546,
    0.21425879001617432
  ],
  "diff": [
    0.5277286767959595,
    -0.19620391726493835,
    -0.7283022403717041,
    -0.011020521633327007
  ],
  "irt_model": "1pl",
  "item_ids": {
    "0": "q4",
    "1": "q3",
    "2": "q1",
    "3": "q2"
  },
  "subject_ids": {
    "0": "pedro",
    "1": "burt",
    "2": "ken",
    "3": "pinguino"
  }
}


In [11]:
!jq . test-1pl/parameters.json

{
  "ability": [
    0.22013945877552032,
    -1.2920401096343994,
    1.3400511741638184,
    0.042264051735401154
  ],
  "diff": [
    0.8419020771980286,
    -0.23540474474430084,
    -1.0303592681884766,
    -0.04312527924776077
  ],
  "irt_model": "1pl",
  "item_ids": {
    "0": "q4",
    "1": "q3",
    "2": "q1",
    "3": "q2"
  },
  "subject_ids": {
    "0": "pedro",
    "1": "burt",
    "2": "ken",
    "3": "pinguino"
  }
}
